# FanCI Tutorial

FanCI is a Python library for computing various post-Hartree-Fock methods
(of the Flexible Ansätze for N-electron CI, or "FanCI" type) using the PyCI library.

Here we will look at the following wavefunction models using **Be** as our toy model:
* AP1roG
* pCCD+S

Lets start showing how to run an AP1roG computation.

In [1]:
# Import modules
import pyci
from pyci.fanci import AP1roG, pCCDS
# optional
import numpy as np
from pyci.test import datafile

Define our system

In [2]:
# System information
filename = datafile("lih_sto6g.fcidump")
ham = pyci.hamiltonian(filename)
e_dict = {}

Compute the Hartree-Fock energy for comparison

In [3]:
# Get Hartree-Fock energy
hf_wfn = pyci.doci_wfn(ham.nbasis, 2, 2)
hf_wfn.add_hartreefock_det()
hf_op = pyci.sparse_op(ham, hf_wfn)
e_dict["HF"] = hf_op.solve(n=1)[0][0] - ham.ecore

Optimize the AP1roG wave function

In [4]:
# Initialize AP1roG instance
ap1rog = AP1roG(ham, 2)

# Make initial guess
ap1_params = np.zeros(ap1rog.nparam, dtype=pyci.c_double)
ap1_params[-1] = e_dict["HF"]

# Optimize wavefunction
ap1rog_results = ap1rog.optimize(ap1_params, use_jac=True)
e_dict["AP1roG"] = ap1rog_results.x[-1]

Use the AP1roG wave function to generate an initial guess for the pCCD+S wave function; optimize it

In [5]:
# Initialize pCCD+S instance
pccds = pCCDS(ham, 2, 2)

# Make initial guess from AP1roG params
pccds_params = np.zeros(pccds.nparam, dtype=pyci.c_double)
pccds_params[:pccds.wfn.nocc_up * pccds.wfn.nvir_up] = ap1rog_results.x[:-1]
pccds_params[-1] = ap1rog_results.x[-1]

# Optimize wavefunction
pccds_results = pccds.optimize(pccds_params, use_jac=False)
e_dict["pCCD+S"] = pccds_results.x[-1]

Compare the energies and verify that  $E_\text{HF}$ > $E_\text{AP1roG}$ > $E_\text{pCCD+S}$

In [6]:
# Print energies from various methods
print(f"{'METHOD':>8s}, {'ENERGY':>16s}")
for name, energy in e_dict.items():
    print(f"{name:>8s}, {energy:>16.9e}")

  METHOD,           ENERGY
      HF, -8.947289175e+00
  AP1roG, -8.963531034e+00
  pCCD+S, -8.963613544e+00


Use a larger projection space (seniority-(0,2) + excitation-(0,1,2,3) CI) to approximately evaluate the pCCD+S RDMs

In [7]:
# Make larger projection space over which to evaluate RDMs
proj_wfn = pyci.fullci_wfn(ham.nbasis, 2, 2)
# Add seniority 0 and 2 determinants
pyci.add_seniorities(proj_wfn, 0, 2)
# Add singly, doubly, triply excited determinants
pyci.add_excitations(proj_wfn, 0, 1, 2, 3)
# Evaluate coefficients with optimized pCCD+S
coeffs = pccds.compute_overlap(pccds_results.x[:-1], proj_wfn.to_occ_array())

# Compute RDMs using larger projection space and pCCD+S coefficients
d1, d2 = pyci.compute_rdms(proj_wfn, coeffs)
rdm1, rdm2 = pyci.spinize_rdms(d1, d2)

Validate the RDMs; ensure that  $\text{tr}\left(\gamma\right) \approx n_\text{elec}$  and that $\frac{1}{2}\sum_{pq}{\Gamma_{pqpq}} \approx n_\text{pair}$

In [8]:
# Validate the RDMs
nelec = np.trace(rdm1)
npairs = np.einsum('pqpq', rdm2) / 2.0
print(f"Number of electrons = {nelec:.1f}")
print(f"Number of pairs     = {npairs:.1f}")

Number of electrons = 4.1
Number of pairs     = 6.1
